In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
from mpl_toolkits.mplot3d import Axes3D     
from matplotlib import cm

class DataPlotter:
    def __init__(self, figsize):
        self.figsize = figsize
        self.fig = None
        self.ax = None
        self.plots = []
        self.subplot_kw = {}
        
    def plot_scalar_field(self, grid, scalar_field, **kwargs):
        pass
    
    def plot_vector_field(self, grid, scalar_field, **kwargs):
        pass
    
    def remove_plots(self):
        for p in self.plots:
            p.remove()    
        
        self.plots = []
    
    def close(self):
        if self.fig is None:
            return
        
        plt.show()
        plt.close()
            
        self.remove_plots()
        self.fig = None
        self.ax = None
        
    def plot_point(self, point, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize, **self.subplot_kw)
            
        p = self.ax.scatter(*point, **kwargs)
        self.plots.append(p)
        
        return self.plots[-1]
    
    def plot_curve(self, curve, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize, **self.subplot_kw)
            
        p, = self.ax.plot(*curve, **kwargs)
        self.plots.append(p)
        
        return self.plots[-1]
        
        
class DataPlotter1D(DataPlotter):
    def __init__(self, figsize=(10,10)):
        DataPlotter.__init__(self, figsize)
        #self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize)
        
        self.colors = matplotlib.cm.get_cmap("tab10")
        self.colors = self.colors(np.linspace(0.0, 1.1, 11, endpoint=False))    
        
    def plot_scalar_field(self, grid, scalar_field, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize)
            
        try: kwargs.pop("cmap")
        except: pass
        try: kwargs.pop("edgecolor")
        except: pass
        
        try: kwargs["color"]
        except: kwargs["color"] = self.colors[len(self.plots)%10]
        
        p, = self.ax.plot(grid[0][:-1], scalar_field, **kwargs)
        self.plots.append(p)
                         
        return self.plots[-1]
    
    def plot_vector_field(self, grid, vector_field, **kwargs):
        return self.plot_scalar_field(grid, vector_field[0], **kwargs)       
        
class DataPlotter2D(DataPlotter):
    def __init__(self, figsize=(10,10)):
        DataPlotter.__init__(self, figsize)
        #self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize)
        
    def plot_scalar_field(self, grid, scalar_field, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize)
        
        try: kwargs["cmap"]
        except: kwargs["cmap"] = "rainbow"

        try: kwargs["edgecolor"]
        except: kwargs["edgecolor"] = "black"
            
        p = self.ax.pcolor(*grid, scalar_field.transpose(), **kwargs)
                           
        self.plots.append(p)
        
        return self.plots[-1]
    
    def plot_vector_field(self, grid, vector_field, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize)
            
        x,y = np.meshgrid(0.5*(grid[0][1:]+grid[0][:-1]), 0.5*(grid[1][1:]+grid[1][:-1]))
        p = self.ax.quiver(x, y, (vector_field[0]).transpose(), (vector_field[1]).transpose(), **kwargs, scale=0.3, scale_units='inches')
        
        self.plots.append(p)
        
        return self.plots[-1]
        

class DataPlotter3D(DataPlotter):
    
    def __init__(self, figsize=(10,10)):
        DataPlotter.__init__(self, figsize)
        self.subplot_kw = {"subplot_kw" : {"projection" : "3d"}}
        #self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize, **self.subplot_kw)
        
    def plot_scalar_field(self, grid, scalar_field, **kwargs):
        
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize, **self.subplot_kw)
            
        try: kwargs["alpha"]
        except: kwargs["alpha"] = 0.7
            
        try: kwargs["cmap"]
        except: kwargs["cmap"] = "rainbow"
            
        try: kwargs["edgecolors"]
        except: kwargs["edgecolors"] = "black"
            
        cmap = matplotlib.cm.get_cmap(kwargs["cmap"])
        
        try: 
            vmin = kwargs["vmin"]
            kwargs.pop("vmin")
        except: vmin = np.amin(scalar_field)
            
        try: 
            vmax = kwargs["vmax"] 
            kwargs.pop("vmax")
        except: vmax = np.amax(scalar_field)
        x, y, z = np.meshgrid(grid[0], grid[1], grid[2])
        d = scalar_field.transpose((1,0,2))
        if vmin == vmax:
            p = self.ax.voxels(x, y, z, np.where(~np.isnan(d), True, False), **kwargs, facecolors=cmap(0.5))

        else:
            p = self.ax.voxels(x, y, z, np.where(~np.isnan(d), True, False), **kwargs, facecolors=cmap((d-vmin)/(vmax-vmin)))  
            
        p = list(p.values())

        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        m = cm.ScalarMappable(cmap=cmap, norm=norm)
        m.set_array([])
        
        self.plots += p
        
        return self.plots[-1]
    
    def plot_vector_field(self, grid, vector_field, **kwargs):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(1, 1, figsize=self.figsize, **self.subplot_kw)
            
        #try:
        #scalar_field = kwargs["scalar_field"]
        #kwargs.pop("scalar_field")
        #try: 
        #    vmin = kwargs["vmin"]
        #    kwargs.pop("vmin")
        #except: vmin = np.amin(scalar_field)

        #try: 
        #    vmax = kwargs["vmax"] 
        #    kwargs.pop("vmax")
        #except: vmax = np.amax(scalar_field)

        #cmap = matplotlib.cm.get_cmap(kwargs["cmap"])

        #kwargs["colors"] = list(cmap(((scalar_field.transpose((1,0,2))).flatten()-vmin)/(vmax-vmin)))
        #print(kwargs["colors"])
        #except:
        #   pass

        x,y,z = np.meshgrid(0.5*(grid[0][1:]+grid[0][:-1]), 0.5*(grid[1][1:]+grid[1][:-1]), 0.5*(grid[2][1:]+grid[2][:-1]))
        p = self.ax.quiver(x, y, z, (vector_field[0]).transpose((1,0,2)), (vector_field[1]).transpose((1,0,2)), (vector_field[2]).transpose((1,0,2)), colors="black", **kwargs)
        
        self.plots.append(p)
        
        return self.plots[-1]    
        

In [ ]:
import itertools

class DiscreteVectorCalculus:
    def __init__(self, grid, boundary):
        self.shape = []
        for axis in grid:
            self.shape.append(len(axis)-1)
            
        if list(boundary.shape) != self.shape:
            raise Exception("Shape of boundary, "+str(boundary.shape)+", does not match shape of grid, "+str(self.shape))
            
        self.D = len(self.shape)
        self.boundary = boundary
        self.grid = grid    
        
        self.gradient = []
        for i in range(self.D):
            self.gradient.append(self.get_gradient_i(i))

        self.gradient2 = []
        for i in range(self.D):
            self.gradient2.append(self.operator_on_operator(self.gradient[i], i, self.gradient[i], i))   
        
    def operator_on_field(self, operator, axis, field):
        new_field = np.zeros(field.shape)

        # keep track of axis permutations done by numpy routines
        permutation = [*range(self.D), axis, *range(self.D)]

        # apply i-th gradient to scalar field (matrix multiplication along last axis (D) of graient and i-th axis of field)
        tmp = np.tensordot(operator, field, axes=(self.D, axis))

        # D-th axis of gradient and i-th axis of scalar have been contracted
        permutation.pop(self.D)
        permutation.pop(self.D+axis)

        # now remove redundant axis left from tensor dot

        # create list of axis other than the i-th (redundant axis correspond to these axes)
        other = list(range(self.D))
        other.remove(axis)


        for o in other:
            # find out the remaining doubled axes
            doubles = [d for d, e in enumerate(permutation) if e == o]

            # take only diagonal elements
            tmp = np.diagonal(tmp, axis1 = doubles[0], axis2 = doubles[1])

            # diagonal removes original axis and appends a new axis at the end
            permutation.pop(doubles[1])
            permutation.pop(doubles[0])
            permutation.append(o)

        # transpose routine needs the inverted permutation to restore the original shape
        inv_permutation = permutation[:]
        for i, p in enumerate(permutation):
            inv_permutation[p] = i

        tmp = tmp.transpose(tuple(inv_permutation))

        new_field = tmp

        return new_field 

    def operator_on_operator(self, operator1, axis1, operator2, axis2):
        new_operator = np.zeros(operator2.shape)

        # keep track of axis permutations done by numpy routines
        permutation = [*range(self.D), axis1, *range(self.D), axis2]
        #print(permutation)

        # apply i-th gradient to scalar field (matrix multiplication along last axis (D) of graient and i-th axis of field)
        tmp = np.tensordot(operator1, operator2, axes=(self.D, axis1))

        # D-th axis of gradient and i-th axis of scalar have been contracted
        permutation.pop(self.D)
        permutation.pop(self.D+axis1)

        # now remove redundant axis left from tensor dot

        # create list of axis other than the i-th (redundant axis correspond to these axes)
        other = list(range(self.D))
        other.remove(axis1)


        for o in other:
            # find out the remaining doubled axes
            doubles = [d for d, e in enumerate(permutation) if e == o]

            if len(doubles) > 2:
                doubles = doubles[:-1]

            # take only diagonal elements
            tmp = np.diagonal(tmp, axis1 = doubles[0], axis2 = doubles[1])

            # diagonal removes original axis and appends a new axis at the end
            permutation.pop(doubles[1])
            permutation.pop(doubles[0])
            permutation.append(o)

        # find the extra occurence of axis2 (the axis on which the new operator is acting) and place it at the end
        doubles = [d for d, e in enumerate(permutation) if e == axis2]
        permutation[doubles[1]] = self.D

        # transpose routine needs the inverted permutation to restore the original shape
        inv_permutation = permutation[:]
        for i, p in enumerate(permutation):
            inv_permutation[p] = i


        tmp = tmp.transpose(tuple(inv_permutation))
        new_operator = tmp

        return new_operator

    def get_gradient_i(self, axis):

        g = np.zeros((*self.shape, self.shape[axis]))

        other = list(range(self.D))
        other.remove(axis)

        others = []
        for o in other:
            others.append(list(range(self.shape[o])))

        this = list(range(self.shape[axis]))


        for combination in itertools.product(*others,this):
            indices = [0]*self.D
            for i, o in enumerate(other):
                indices[o] = combination[i]
            indices[axis] = combination[-1]

            if self.boundary[tuple(indices)]:
                continue

            left = indices[:]
            left[axis] -= 1

            cond = False
            if left[axis] >= 0:
                cond = self.boundary[tuple(left)]
            else:
                cond = True             

            if cond:
                delta = self.grid[axis][indices[axis]+1] - self.grid[axis][indices[axis]]
                g[(*indices, indices[axis]+1)] = 1.0/delta
                g[(*indices, indices[axis])] = -1.0/delta
                continue

            right = indices[:]
            right[axis] += 1

            cond = False
            if right[axis] < shape[axis]:
                cond = self.boundary[tuple(right)]
            else:
                cond = True 

            if cond:
                delta = self.grid[axis][indices[axis]] - self.grid[axis][indices[axis]-1]
                g[(*indices, indices[axis])] = 1.0/delta
                g[(*indices, indices[axis]-1)] = -1.0/delta  
                continue            

            delta = self.grid[axis][indices[axis]+1] - self.grid[axis][indices[axis]-1] 
            g[(*indices, indices[axis]-1)] = -1.0/delta
            g[(*indices, indices[axis]+1)] = 1.0/delta

        return g
    
    def grad(self, scalar_field):
        g = np.zeros((self.D, *scalar_field.shape))

        for i in range(self.D):
            g[i] = self.operator_on_field(self.gradient[i], i, scalar_field)

        return g


    def div(self, vector_field):
        d = np.zeros(vector_field.shape[1:])

        for i in range(self.D):
            d += self.operator_on_field(self.gradient[i], i, vector_field[i])

        return d

    def laplace(self, vector_field):
        l = np.zeros(vector_field.shape)
        for i in range(self.D):
            for j in range(self.D):   
                l[i] += self.operator_on_field(self.gradient2[j], j, vector_field[i])

        return l   
    
    def curl(self, vector_field):
        if self.D == 1:
            c = np.zeros(vector_field.shape[1:])
        elif self.D == 2:
            c = np.zeros(vector_field.shape[1:])

            c += self.operator_on_field(self.gradient[0], 0, vector_field[1])
            c -= self.operator_on_field(self.gradient[1], 1, vector_field[0])
            
        else:
            c = np.zeros(vector_field.shape)

        return c      
    
 

In [ ]:
from scipy.integrate import solve_ivp

class OdeSolver:
    def __init__(self, fluid_model):
        self.fluid_model = fluid_model
        
    def update(self, dt, n_steps=1):
        D = self.fluid_model.D
        shape = self.fluid_model.shape
        
        def dudt(t, u4):
            dudt = np.zeros((D+1, *shape))
            dudt[0], dudt[1:] = self.fluid_model.get_time_tendencies()

            return dudt.flatten()
        
        u4 = np.zeros((D+1, *shape))
        u4[0] = self.fluid_model.rho
        u4[1:] = self.fluid_model.u

        t_eval = np.linspace(0.0, n_steps*dt, n_steps)
        sol = solve_ivp(dudt, [0.0, t_eval[-1]], u4.flatten(), t_eval=t_eval, method='LSODA')
        u4 = sol.y[:,-1]
        u4 = u4.reshape((D+1, *shape))

        self.fluid_model.rho = u4[0]
        self.fluid_model.u = u4[1:]        
        

In [ ]:
import itertools

class FluidModel:
    def __init__(self, grid, bottom, rho, u, **parameters):
        
        self.grid = grid
        
        self.shape = []
        for axis in self.grid:
            self.shape.append(len(axis)-1)
            
        self.D = len(self.shape)
            
        if list(bottom.shape) != self.shape:
            raise Exception("Shape of bottom, "+str(bottom.shape)+", does not match shape of grid, "+str(self.shape))
        self.bottom = bottom
            
        if list(rho.shape) != self.shape:
            raise Exception("Shape of density, "+str(rho.shape)+", does not match shape of grid, "+str(self.shape))
        self.rho = rho
        
        if list(u.shape) != [self.D, *self.shape]:
            raise Exception("Shape of density, "+str(rho.shape)+", does not match shape of grid, "+str(self.shape))
        self.u = u            
            
        
        if self.D == 1:
            self.plotter = DataPlotter1D()
        elif self.D == 2:
            self.plotter = DataPlotter2D()
        elif self.D == 3:
            self.plotter = DataPlotter3D()
            
        self.create_bottom2()
        
        self.dvc = DiscreteVectorCalculus(self.grid, self.bottom)
        
        self.grad = self.dvc.grad
        self.div = self.dvc.div
        self.curl = self.dvc.curl
        self.laplace = self.dvc.laplace
        
        try: self.nu = parameters["nu"]
        except: self.nu = 0.0
        
        try: 
            self.omega = parameters["omega"]
            if self.D == 1:
                print("Warning setting omega in 1D has no effect!")
            elif self.D == 2 and type(self.omega) is not float:
                raise Exception("In 2D, omega can only be scalar but is "+str(self.omega))
            elif self.D == 3 and len(self.omega) != 3:   
                raise Exception("Dimension of omega does not match shape.")
        except: 
            if self.D < 3:
                self.omega = 0.0
            else:
                self.omega = np.zeros((self.D))
                
        
        self.solver = OdeSolver(self)
        
        
    def advection(self):
        ad = np.zeros((self.D, *self.shape))

        for i in range(self.D):
            tmp = self.grad(self.u[i])
            for j in range(self.D):
                ad[i] -= self.u[j]*tmp[j]

        return ad

    def body_forces(self):
        f_b = np.zeros((self.D, *self.shape))
        return f_b

    def pressure_force(self):

        f_r = np.zeros((self.D, *self.shape))
        return f_r

    def friction_force(self):
        nu = self.nu

        f_a = np.zeros((self.D, *self.shape))
        f_a = nu*self.laplace(self.u)

        for i in range(self.D):
            f_a[i] = np.divide(f_a[i], self.rho, out=np.zeros_like(self.rho), where=self.rho!=0)

        return f_a

    def apparent_forces(self):

        f_ap = np.zeros((self.D, *self.shape))
        if self.D == 2:
            f_ap[0] += self.omega**2*np.tensordot(self.grid[0][:-1], np.ones(self.shape[1]), axes=0)
            f_ap[1] += self.omega**2*np.tensordot(np.ones(self.shape[0]), self.grid[1][:-1], axes=0)

            f_ap[0] += 2.0*self.omega*self.u[1]
            f_ap[1] += -2.0*self.omega*self.u[0]

        return f_ap   
    
    def get_time_tendencies(self):
        
        from scipy.ndimage import gaussian_filter
        
        drho_dt = np.zeros(tuple(self.shape))
        du_dt = np.zeros((D, *self.shape))
        
        for i in range(self.D):
            #self.u[i] = gaussian_filter(self.u[i], sigma=0.1) # smooth the velocity field a little bit to damp out high modes (sound waves)
            self.u[i] = np.where(self.bottom2 == False, self.u[i], 0.0) # apply no-splip condition
        
        # mass continuity equation
        drho_dt = -self.div(self.rho*self.u)
        
        #g_rho_smooth = self.grad(gaussian_filter(self.rho, sigma=0.1))
        #for i in range(self.D):
        #    drho_dt -= self.u[i]*g_rho_smooth[i]

        # momentum equation
        du_dt = self.advection()
        du_dt += self.body_forces()
        du_dt += self.pressure_force()
        
        if self.nu != 0.0:
            du_dt += self.friction_force()
            
        if np.linalg.norm(self.omega) != 0.0:
            du_dt += self.apparent_forces()
            
        return drho_dt, du_dt
    
    def update(self, dt, n_steps=1):
    
        self.solver.update(dt, n_steps)

        # apply no-slip condition to updated velocity field
        for i in range(self.D):
            self.u[i] = np.where(self.bottom2 == False, self.u[i], 0.0)

            
    def create_bottom2(self):
        """Create no-slip layer directly above the bottom."""
        import copy

        # initialize layer, called bottom2, with bottom
        self.bottom2 = copy.copy(self.bottom)
        
        # go over all axes
        for axis in range(self.D):
            
            # create list of axes other than axis
            other = list(range(self.D))
            other.remove(axis)

            # create list of indices for other axes
            others = []
            for o in other:
                others.append(list(range(self.shape[o])))

            # create list of indices for this axis
            this = list(range(self.shape[axis]))

            # go over all combination of indices
            for combination in itertools.product(*others,this):
                # create list of indices for this combination
                indices = [0]*self.D
                for i, o in enumerate(other):
                    indices[o] = combination[i]
                indices[axis] = combination[-1]

                # if we are at the bottom there is nothing todo since bottom2 is initialized with bottom values
                if self.bottom[tuple(indices)]:
                    continue

                # look at the grid cell left of us
                left = indices[:]
                left[axis] -= 1

                cond = False
                if left[axis] >= 0:
                    cond = self.bottom[tuple(left)]         

                # if left of us is bottom, this grid cell is subjected to the no-slip condititon
                if cond:
                    self.bottom2[tuple(indices)] = True
                    continue

                # look at the grid cell right of us
                right = indices[:]
                right[axis] += 1

                cond = False
                if right[axis] < shape[axis]:
                    cond = self.bottom[tuple(right)]

                # if right of us is bottom, this grid cell is subjected to the no-slip condititon
                if cond:
                    self.bottom2[tuple(indices)] = True
                    continue    
                    
                  

In [ ]:
class MyFluidModel(FluidModel):
    def __init__(self, grid, bottom, rho, u, **parameters):
        FluidModel.__init__(self, grid, bottom, rho, u, **parameters)
        
    def body_forces(self):
        """Gravitation in negative direction of the last axis"""
        f_b = np.zeros((self.D, *self.shape))

        #g = 9.81
        #f_b[-1] = -g

        return f_b

    def pressure_force(self):
        """Pressure is proportional to density ~ ideal gas with constant temperature"""
        a = 10.0
        p = a*self.rho

        f_r = -self.grad(p)
        for i in range(self.D):
            f_r[i] = np.divide(f_r[i], self.rho, out=np.zeros_like(self.rho), where=self.rho!=0) 


        return f_r

In [ ]:
# 1. define our spatial discretization
# 3D
axes = ["x", "y", "z"]
shape = [10, 8, 5]

# 2D
axes = ["x", "y"]
shape = [20, 15]

# 1D
#axes = ["x"]
#shape = [30]

# our dimension is consequently
D = len(shape)

# discretize the interval [0, 1] according to our shape
grid = []
for i in range(D):
    grid.append(np.linspace(0.0, 1.0, shape[i]+1))
    print(grid[i])


# 2. define the boundaries of our model
bottom = np.full(shape=shape, fill_value=False)


if D == 1:
    bottom[0] = True # left wall
    bottom[-1] = True # right wall
if D == 2:
    bottom[0,:] = True # walls
    bottom[-1,:] = True  # walls
    bottom[0:(2*shape[0])//3, shape[1]//2] = True # barrier
    bottom[:,-1] = True # closed top
    bottom[:,0] = True # closed bottom
elif D == 3:
    bottom[:,:,0] = True # bottom wall
    bottom[0,:,:] = True # left wall in x direction
    bottom[-1,:,:] = True # right wall in x direction
    bottom[:,0,:] = True # left wall in x direction
    bottom[:,-1,:] = True  # right wall in x direction
    

# 3. initialize velocity field as vector field
u = np.zeros((D, *shape))

# 4. initialize density as scalar field
rho = np.zeros(tuple(shape))
if D == 1:
    rho += grid[0][:-1]+0.2 # ~ inverted hydrostatic balance
if D == 2:
    rho += np.tensordot(np.ones((shape[0])), np.exp(-0.8*(grid[1][-1]-grid[1][:-1])), axes=0) # ~ inverted hydrostatic balance
if D == 3:
    rho += np.tensordot(np.tensordot(np.ones((shape[0])), np.ones((shape[1])), axes=0), np.exp(-0.8*(grid[2][:-1])), axes=0) # ~ hydrostatic balance

# set density at boundary to zero
rho = np.where(~bottom, rho, 0.0)    
    
# 5. initialize fluid model
fm = MyFluidModel(grid, bottom, rho, u, nu=0.2, omega=0.0)

# 6. plot the bottom (boundary)
fm.plotter.plot_scalar_field(fm.grid, fm.bottom.astype(float))
fm.plotter.close()

# 7. create a tracer particle
if D == 1:
    r = np.array([0.5*(grid[0][int(0.8*shape[0])-1]+grid[0][int(0.8*shape[0])]), 0.0])
if D == 2:
    r = np.array([0.5*(grid[0][int(0.8*shape[0])-1]+grid[0][int(0.8*shape[0])]), 0.5*(grid[1][int(0.8*shape[1])-1]+grid[1][int(0.8*shape[1])])])
if D == 3:
    r = np.array([0.5*(grid[0][int(0.8*shape[0])-1]+grid[0][int(0.8*shape[0])]), 0.5*(grid[1][int(0.8*shape[1])-1]+grid[1][int(0.8*shape[1])]), 0.5*(grid[2][int(0.8*shape[2])-1]+grid[2][int(0.8*shape[2])])])

# 8. plot the initial state of our system 
fm.plotter.plot_scalar_field(fm.grid, fm.rho)
fm.plotter.plot_vector_field(fm.grid, fm.u)
fm.plotter.plot_point(r, color="grey")
fm.plotter.close()

In [ ]:
%matplotlib

fm.plotter.close()

# take a small time step (sound waves are still in)
dt = 0.00001

# store the trajectory of the tracer particle
traj = [0]*fm.D
for i in range(fm.D):
    traj[i] = [r[i]]

# propagate the fluid model in time
for t in range(10000):
    

    # plot current state of fluid model
    if fm.D == 3:
        fm.plotter.plot_scalar_field(fm.grid, fm.rho, vmin=0, vmax=1, cmap="rainbow", alpha=0.2, edgecolors=None)
        fm.plotter.plot_vector_field(fm.grid, fm.u)
    if fm.D == 2:
        fm.plotter.plot_scalar_field(fm.grid, fm.rho, vmin=0, vmax=1, cmap="rainbow")
        fm.plotter.plot_vector_field(fm.grid, fm.u)
    if fm.D == 1:
        fm.plotter.plot_scalar_field(fm.grid, fm.rho)
        fm.plotter.plot_vector_field(fm.grid, fm.u)

    # plot the tracer particle 
    fm.plotter.plot_point(r, color="grey")
    if fm.D != 1:
        fm.plotter.plot_curve(traj, color="grey")
    plt.draw()
    plt.pause(0.01)

    # make space for new plot
    fm.plotter.remove_plots()
    
    # integrate 100 small time steps forward 
    solve_period = 100
    fm.update(dt, solve_period)
    
    # move the tracer particle with updated field
    
    #   find the grid cell where the particle is located
    idx = [0]*fm.D
    for i in range(fm.D):
        idx[i] = (np.abs(fm.grid[i] - r[i])).argmin()
    
    #   update the tracer position with corresponding velocity at its position
    for i in range(fm.D):
        try:
            r[i] += fm.u[i][tuple(idx)]*solve_period*dt
        except:
            pass
        traj[i].append(r[i])
